In [1]:
import pandas as pd
import random

In [2]:
TRAIN=False

In [3]:
results = pd.read_csv('results.csv')
results.loc[:, 'mask'] = results.loc[:, 'mask'].str.replace('X', '')
roster = pd.read_csv('roster.csv').set_index('idplayer')
tournaments = pd.read_csv('tournaments.csv').set_index('idtournament')
tournaments = tournaments.loc[~tournaments.questions_total.isnull(), :]
common_tours = list(set(tournaments.index).intersection(set(results.idtournament)))
results = results.loc[results.idtournament.isin(common_tours), :]
tournaments = tournaments.loc[common_tours, :]

q_total = pd.DataFrame(results.groupby('idtournament').first()\
                       .loc[:, 'mask'].str.len()).dropna().rename(columns={'mask': 'questions_total'})
tournaments = pd.merge(tournaments.drop(columns=['questions_total']), q_total, left_index=True, right_index=True)

In [4]:
tournaments.head(1)

,name,town,long_name,date_start,date_end,tour_count,tour_questions,tour_ques_per_tour,type_name,main_payment_value,discounted_payment_value,discounted_payment_reason,date_requests_allowed_to,comment,site_url,main_payment_currency,discounted_payment_currency,questions_total
idtournament,,,,,,,,,,,,,,,,,,
2,Летние зори,Саранск,NaN,2003-08-09 00:00:00,2003-08-09 00:00:00,3.0,15.0,NaN,Обычный,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0


In [5]:
results.head(2)

,idteam,current_name,base_name,position,questions_total,mask,bonus_a,bonus_b,tech_rating,predicted_position,d_bonus_a,d_bonus_b,d_diff_bonus,included_in_rating,idtournament,diff_bonus
0,209,Команда Коваленко,Команда Коваленко,11.0,34,1111101000101111001100011000111101001111001110...,2230.0,-87.0,4509.0,7.0,2230,663,-87.0,1.0,3236,NaN
1,270,Синоп,Синоп,14.5,31,1110110000101100001100101101101101010011010000...,2189.0,119.0,2567.0,20.0,2189,618,119.0,1.0,3236,NaN


In [6]:
roster.head(1)

,is_captain,is_base,is_foreign,idteam,idtournament
idplayer,,,,,
19298,1,1,0,209,3236


In [7]:
def skill_init():
    return random.uniform(0, 1)
if TRAIN:
    roster.loc[:, 'skill'] = None
    roster.loc[:, 'skill'] = roster.loc[:, 'skill'].apply(lambda x: skill_init())
    roster.head(1)
else:
    roster = pd.read_csv('skills.csv')
# player_skills_dict = {id: skill_init() for id in roster.idplayer}
# pd.DataFrame(player_skills_dict, index=[0]).T.head()

In [9]:
roster.head(1)

,idplayer,is_captain,is_base,is_foreign,idteam,idtournament,skill
0,19298,1,1,0,209,3236,-0.600313


In [10]:
def difficulty_init():
    return random.uniform(0, 1)
if TRAIN:
    question_difficulty = {f"{str(tourn)}_{str(ques)}": difficulty_init() for tourn in tournaments.index 
                       for ques in range(int(tournaments.loc[tourn, 'questions_total']))}
else:
    question_difficulty = pd.read_csv('qus_dif.csv')
    question_difficulty = dict(zip(question_difficulty.iloc[:, 0], question_difficulty.iloc[:, 1]))
pd.DataFrame(question_difficulty, index=[0]).T.head()

In [15]:
def sigmoid(x):
    return 1 / (1 + pd.np.exp(-x))    

**Model**<br>
$p(y=1| S_{team}, \theta) = \sigma(S_{team} - \theta)$ <br>
$S_{team}$ - team skill<br>
$S_{team} = S_{player\_1} + S_{player\_2} + ... + S_{player\_N}$<br>
$\theta_j$ - difficulty of question j<br>
y - team answer, 1 for correct, 2 for incorrect<br>
$p(y_{hat}=y_{real}| S_{team}, \theta) = \sigma(S_{team} - \theta)^{y_{real}} * (1-\sigma(S_{team} - \theta))^{(1-y_{real})}$<br>
<br>
$LikelyHood = \frac{1}{N}\prod_{i=1}^{N} p(y_{hat\_i}=y_{real\_i}| S_{team}, \theta)$<br>
$log(LikelyHood) = LL = \frac{1}{N}\sum_{i=1}^{N} log(p(y_{hat\_i}=y_{real\_i}| S_{team}, \theta))=
y_{real}*log(\sigma(S_{team} - \theta)) + (1-y_{real})* log(1-\sigma(S_{team} - \theta))$<br>
<br>
$-log(LikelyHood) \rightarrow min$ w.r.t. $S_{player\_i}$ and $\theta_j$

**Derivatives:**

$\sigma'(x) = \sigma(x)*(1-\sigma(x))$

$\frac{\partial LL}{\partial \theta} = \frac{y_{real}}{\sigma(S_{team} - \theta)} * \sigma'(S_{team} - \theta) * (-1) +
\frac{1-y_{real}}{1-\sigma(S_{team} - \theta)} * (-\sigma'(S_{team} - \theta)) * (-1) = 
-y_{real}*(1-\sigma(S_{team} - \theta)) + (1-y_{real})*\sigma(S_{team} - \theta)
$<br>

$\frac{\partial LL}{\partial S_i} = \frac{y_{real}}{\sigma(S_{team} - \theta)} * \sigma'(S_{team} - \theta) +
\frac{1-y_{real}}{1-\sigma(S_{team} - \theta)} * (-\sigma'(S_{team} - \theta)) = 
y_{real}*(1-\sigma(S_{team} - \theta)) - (1-y_{real})*\sigma(S_{team} - \theta)$<br>


**Short derivatives:**

$\frac{\partial -LL}{\partial \theta} = +y_{real}*(1-\sigma(S_{team} - \theta)) - (1-y_{real})*\sigma(S_{team} - \theta)
$<br>

$\frac{\partial -LL}{\partial S_i} = -y_{real}*(1-\sigma(S_{team} - \theta)) + (1-y_{real})*\sigma(S_{team} - \theta)$<br>


**Gradient update**<br>
$S_{i\_new} = S_{i\_old} - learning\_rate * \frac{\partial LL}{\partial S_i}$<br>
$\theta_{new} = \theta_{old} - learning\_rate * \frac{\partial LL}{\partial \theta}$<br>

In [27]:
from tqdm import tqdm_notebook, tqdm
tqdm.pandas()

In [62]:
def gd(lr=0.01, epochs=10, steps_per_tour = 10, certain_tour=None):
    fails=0
    fail_list = []
    for _ in tqdm_notebook(range(epochs)):
#     for _ in range(epochs):
        # get random tournament, team and question

        if certain_tour:
            tour = certain_tour
        else:
            tour = random.sample(tournaments.index.tolist(), 1)[0]

        t_subset = results.loc[results.idtournament == tour, ['idteam', 'mask']]
        for _ in tqdm_notebook(range(steps_per_tour)):
#         for _ in range(steps_per_tour):
            team = random.sample(t_subset.idteam.tolist(), 1)[0]
            players = roster.loc[(roster.idteam == team) & (roster.idtournament == tour), :].index.tolist()
            quest = random.randint(0, int(tournaments.loc[tour, 'questions_total'])-1)

            # get question difficulty
            quest_dif_key = f"{str(tour)}_{str(quest)}"
            try:
                difficulty = question_difficulty[quest_dif_key]
            except:
                print('!', end='')
                fails+=1
                fail_list.append(quest_dif_key)
                continue

            skills = roster.loc[(roster.index.isin(players)) &
                                (roster.idteam == team) &
                                (roster.idtournament == tour), 'skill'].tolist()
            team_skill = sum(skills)
            team_sigmoid = sigmoid(team_skill - difficulty)

            # get y (1 for correct answer, 0 for incorrect)
            try:
                y_true = int(t_subset.loc[t_subset.idteam == team, 'mask'].values[0][quest])
            except:
                print(f'Fail for tour {tour}, team {team}, question {quest}')
                continue

            # weights update
            question_difficulty[quest_dif_key] -= lr * (y_true * (1-team_sigmoid) - (1-y_true)*team_sigmoid)
            roster.loc[players, 'skill'] -= lr * (-y_true * (1-team_sigmoid) + (1-y_true)*team_sigmoid) / 6
    print(f"Fails: {fails}")
    return fail_list

In [63]:
def score_one_answer(tour, team, quest):
    quest_dif_key = f"{str(tour)}_{str(quest)}"
    difficulty = question_difficulty[quest_dif_key]
    players = roster.loc[(roster.idteam == team) & (roster.idtournament == tour), :].index.tolist()
    skills = roster.loc[(roster.index.isin(players)) &
                                (roster.idteam == team) &
                                (roster.idtournament == tour), 'skill'].tolist()
    team_skill = sum(skills)
    team_sigmoid = sigmoid(team_skill - difficulty)
    
    y_true = int(results.loc[(results.idtournament == tour) &
                             (results.idteam == team), 'mask'].values[0][quest])
    print(f"Y: {y_true}\nSigmoid: {team_sigmoid}")

In [84]:
fl = gd(lr=0.1, epochs=500, steps_per_tour = 100, certain_tour=None)
pd.DataFrame(question_difficulty, index=[0]).T.to_csv('qus_dif.csv')
roster.to_csv('skills.csv')

Fail for tour 5221, team 67284, question 24
Fail for tour 5221, team 67284, question 34
Fail for tour 5221, team 67284, question 18
Fail for tour 5221, team 53456, question 28


Fail for tour 5097, team 53216, question 13
Fail for tour 5097, team 58738, question 12


Fail for tour 5121, team 53456, question 32


Fail for tour 3589, team 27161, question 103
Fail for tour 3589, team 40247, question 171
Fail for tour 3589, team 27161, question 123
Fail for tour 3589, team 27161, question 179


Fail for tour 4469, team 46049, question 76
Fail for tour 4469, team 48412, question 72


...

Fail for tour 2040, team 3912, question 52


Fail for tour 1858, team 5970, question 43


Fail for tour 1931, team 3880, question 38
Fail for tour 1931, team 8423, question 49
Fail for tour 1931, team 40999, question 71
Fail for tour 1931, team 40369, question 69
Fail for tour 1931, team 39247, question 58


Fail for tour 2345, team 2542, question 41
Fail for tour 2345, team 43995, question 38
Fail for tour 2345, team 38808, question 36
Fail for tour 2345, team 44285, question 37
Fail for tour 2345, team 44510, question 36
Fail for tour 2345, team 40452, question 36
Fail for tour 2345, team 44363, question 40
Fail for tour 2345, team 44510, question 41
Fail for tour 2345, team 39775, question 37
Fail for tour 2345, team 44404, question 38
Fail for tour 2345, team 44365, question 40


Fails: 0


## Total Loss

In [16]:
results.head(2)

,idteam,current_name,base_name,position,questions_total,mask,bonus_a,bonus_b,tech_rating,predicted_position,d_bonus_a,d_bonus_b,d_diff_bonus,included_in_rating,idtournament,diff_bonus
0,209,Команда Коваленко,Команда Коваленко,11.0,34,1111101000101111001100011000111101001111001110...,2230.0,-87.0,4509.0,7.0,2230,663,-87.0,1.0,3236,NaN
1,270,Синоп,Синоп,14.5,31,1110110000101100001100101101101101010011010000...,2189.0,119.0,2567.0,20.0,2189,618,119.0,1.0,3236,NaN


In [29]:
res = results.copy(deep=True)
res = res.iloc[:1000]

In [18]:
roster.head(1)

,idplayer,is_captain,is_base,is_foreign,idteam,idtournament,skill
0,19298,1,1,0,209,3236,-0.600313


In [30]:
# res.loc[:, 't_skill'] = 
# res.loc[:, 't_skill']
# res.loc[:, 't_skill'].apply(lambda x: )

res.loc[:, 't_skill'] = res.progress_apply(lambda row: roster.loc[(roster.idteam == row['idteam']) & 
                                 (roster.idtournament == row['idtournament']), 'skill'].sum(), axis=1)


 40%|███████████████████████████████████████████████████████████▊                                                                                          | 399/1000 [00:04<00:06, 86.46it/s]


 81%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                            | 810/1000 [00:08<00:02, 87.23it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 90.67it/s]

In [33]:
question_difficulty

{'2_0': 1.4901700017603288,
 '2_1': 0.383639555706725,
 '2_2': 0.22276746410583295,
 '2_3': 0.9087809504371764,
 '2_4': 1.4361031167883578,
 '2_5': 1.3412277349691264,
 '2_6': 0.9305425837032798,
 '2_7': 1.0896437968619572,
 '2_8': 0.8415313207966179,
 '2_9': 1.323303574177055,
 '2_10': 0.6182058740326459,
 '2_11': 0.9617339911888964,
 '2_12': 1.1244069982938023,
 '2_13': 1.0614127606508237,
 '2_14': 0.8697426681444864,
 '3_0': 0.1419957878021869,
 '3_1': 0.1430597719381509,
 '3_2': 0.009674613043364144,
 '3_3': 0.5461078306959808,
 '3_4': 0.9667904668088679,
 '3_5': 0.4550745815802663,
 '3_6': 0.6591320607366308,
 '3_7': 0.7302398412541337,
 '3_8': 0.06695010030006665,
 '3_9': 0.7494145362893171,
 '3_10': 0.8469071825473344,
 '3_11': 0.022034374165478376,
 '3_12': 0.971900215887067,
 '3_13': 0.012029307182046313,
 '3_14': 0.4092547150617944,
 '6_0': 0.9537505378482204,
 '6_1': 0.7491330707433701,
 '6_2': 0.9198237508318076,
 '6_3': 0.4225947066317307,
 '6_4': 0.5522679314755586,
 '6_5

In [32]:
res['mask']

0      1111101000101111001100011000111101001111001110...
1      1110110000101100001100101101101101010011010000...
2      1111110000011100000010011111100001000111001000...
3      1101100000001110001011011101111111101111100100...
4      1111010010110110001000011001101100101001000000...
5      1100000010000100000100011101101001101101000000...
6      1110111011111111011110111101111111001111011000...
7      1110101000100000001110000101100111001001000000...
8      1110110111100111101010011111101110101101101010...
9      1100001100001000000100000101000101101111010000...
10     1110100010111000001100001001101101001011011000...
11     1110000110101010011100101111111101101000001010...
12     1110000000100110011110011001101001000101000000...
13     0010000001110101100100001001001101000100100000...
14     1100010111110111001011001001111111101111110110...
15     1110110111110011100110001101111111001111011000...
16     1100000100000000000100010101100001000001001000...
17     111000010010010000111001

In [81]:
score_one_answer(3236, 209, 0)

Y: 1
Sigmoid: 0.7789489497169478


In [82]:
score_one_answer(3236, 209, 10)

Y: 1
Sigmoid: 0.2740432939074542


In [15]:
def sgd(lr=0.01, epochs=10, steps_per_tour = 10, batch=10, certain_tour=None):
    fails=0
    fail_list = []
    for _ in tqdm_notebook(range(epochs)):
#     for _ in range(epochs):
        # get random tournament, team and question

        if certain_tour:
            tour = certain_tour
        else:
            tour = random.sample(tournaments.index.tolist(), 1)[0]

        t_subset = results.loc[results.idtournament == tour, ['idteam', 'mask']]
        for _ in tqdm_notebook(range(steps_per_tour)):
#         for _ in range(steps_per_tour):
            team = random.sample(t_subset.idteam.tolist(), 1)[0]
            players = roster.loc[(roster.idteam == team) & (roster.idtournament == tour), :].index.tolist()
            quest = random.randint(0, int(tournaments.loc[tour, 'questions_total'])-1)

            # get question difficulty
            quest_dif_key = f"{str(tour)}_{str(quest)}"
            try:
                difficulty = question_difficulty[quest_dif_key]
            except:
                print('!', end='')
                fails+=1
                fail_list.append(quest_dif_key)
                continue

            skills = roster.loc[(roster.index.isin(players)) &
                                (roster.idteam == team) &
                                (roster.idtournament == tour), 'skill'].tolist()
            team_skill = sum(skills)
            team_sigmoid = sigmoid(team_skill - difficulty)

            # get y (1 for correct answer, 0 for incorrect)
            y_true = int(t_subset.loc[t_subset.idteam == team, 'mask'].values[0][quest])

            # weights update
            question_difficulty[quest_dif_key] -= lr * (y_true * (1-team_sigmoid) - (1-y_true)*team_sigmoid)
            roster.loc[players, 'skill'] -= lr * (-y_true * (1-team_sigmoid) + (1-y_true)*team_sigmoid) / 6
    print(f"Fails: {fails}")
    return fail_list

,idteam,current_name,base_name,position,questions_total,mask,bonus_a,bonus_b,tech_rating,predicted_position,d_bonus_a,d_bonus_b,d_diff_bonus,included_in_rating,idtournament,diff_bonus
0,209,Команда Коваленко,Команда Коваленко,11.0,34,1111101000101111001100011000111101001111001110...,2230.0,-87.0,4509.0,7.0,2230,663,-87.0,1.0,3236,NaN
1,270,Синоп,Синоп,14.5,31,1110110000101100001100101101101101010011010000...,2189.0,119.0,2567.0,20.0,2189,618,119.0,1.0,3236,NaN
2,358,43,43,20.0,26,1111110000011100000010011111100001000111001000...,2074.0,-100.0,3532.0,9.0,2074,501,-100.0,1.0,3236,NaN
3,482,Нимлот,Нимлот,14.5,31,1101100000001110001011011101111111101111100100...,2189.0,-29.0,3376.0,10.0,2189,618,-29.0,1.0,3236,NaN
4,1948,Doom-Doom,Doom-Doom,17.5,29,1111010010110110001000011001101100101001000000...,2145.0,38.0,2629.0,19.0,2145,557,38.0,1.0,3236,NaN
5,2298,Макароны под плинтусом,Макароны под плинтусом,24.0,21,1100000010000100000100011101101001101101000000...,1721.0,-29.0,2086.0,23.0,1721,311,-29.0,1.0,3236,NaN
6,3213,Дикие бозоны Хиггса,Дикие бозоны Хиггса,3.0,49,1110111011111111011110111101111111001111011000...,0.0,8.0,6161.0,3.0,2290,1197,8.0,1.0,3236,NaN
7,3930,Версий.net,Версий.net,24.0,21,1110101000100000001110000101100111001001000000...,1721.0,20.0,1562.0,25.0,1721,311,20.0,1.0,3236,NaN
8,3951,Eclipse,Eclipse,5.0,44,1110110111100111101010011111101110101101101010...,2280.0,-50.0,6019.0,4.0,2280,1012,-50.0,1.0,3236,NaN
9,4822,Беспредел,Беспредел,26.5,20,1100001100001000000100000101000101101111010000...,1399.0,69.0,1105.0,28.0,1399,247,69.0,1.0,3236,NaN


In [16]:
fl


[]

In [17]:
question_difficulty['450_13']

0.3050268334131976

In [18]:
results.loc[results.idtournament==450, 'mask'].str[0]

193447    0
193448    0
193449    0
193450    0
193451    0
193452    0
193453    0
193454    0
193455    0
193456    0
193457    0
193458    0
193459    0
193460    0
193461    0
193462    0
193463    0
193464    0
193465    0
193466    0
193467    0
193468    0
193469    0
193470    0
193471    0
193472    0
193473    0
193474    0
193475    0
193476    0
Name: mask, dtype: object